# Recipe classification

Imagine a colaborative recipe website which wants to propose an automatic tag function which categorize a proposed recipe among a set of recipe type like french, mexican, italian, korean, .... This functionality will be build thanks to a classifier wich will take as input the ingredients of the recipe and will output the recipe type.

In this exercice we will build such a classifier using a set of labelled recipes.

In [1]:
import pandas as pd
import numpy as np
import json

## Data preprocessing

First we need to read and encode the dataset for further processing. The "recipe_train.json" contains an array of recipe with a cuisine filed with the recipe cuisine type (a string) and an ingredients fileds with an array of ingredients (strings). We first load the data and build an np.array of cuisine type to predict (y) and a list of recipes ingredients. 

In [2]:
with open("./recipes_train.json") as f:
    data_train=json.load(f)
    y = np.array([recipe["cuisine"] for recipe in data_train])
    xtext = [recipe["ingredients"] for recipe in data_train]

In [3]:
data_train

[{'id': 10259,
  'cuisine': 'greek',
  'ingredients': ['romaine lettuce',
   'black olives',
   'grape tomatoes',
   'garlic',
   'pepper',
   'purple onion',
   'seasoning',
   'garbanzo beans',
   'feta cheese crumbles']},
 {'id': 25693,
  'cuisine': 'southern_us',
  'ingredients': ['plain flour',
   'ground pepper',
   'salt',
   'tomatoes',
   'ground black pepper',
   'thyme',
   'eggs',
   'green tomatoes',
   'yellow corn meal',
   'milk',
   'vegetable oil']},
 {'id': 20130,
  'cuisine': 'filipino',
  'ingredients': ['eggs',
   'pepper',
   'salt',
   'mayonaise',
   'cooking oil',
   'green chilies',
   'grilled chicken breasts',
   'garlic powder',
   'yellow onion',
   'soy sauce',
   'butter',
   'chicken livers']},
 {'id': 22213,
  'cuisine': 'indian',
  'ingredients': ['water', 'vegetable oil', 'wheat', 'salt']},
 {'id': 13162,
  'cuisine': 'indian',
  'ingredients': ['black pepper',
   'shallots',
   'cornflour',
   'cayenne pepper',
   'onions',
   'garlic paste',
   'm

We must encode the ingredients and define the numeric features that will describe our recipes. To do so we will take a classic approach in text processing called bag of words so in our case bag of ingredients. Each recipe will be then associated with a big vectors of zeros and one. Each element of this vector will correspond to an ingredients and we will put a one if the recipe use this ingredients and a zero if it's not the case.

To do so we start by computing the list of all the ingredients and a dict which associated to each ingredients an integer

In [4]:
ingredients = np.unique(np.concatenate(xtext))
dict_ingredients = dict((ingredients[i],i) for i in range(0, len(ingredients)))

We may then encode the recipe using a small function that will take an array of ingredients and return a vector of size (number of possible ingredients) with ones at the right places. Eventually we stack all these vectors in our data matrix X.

In [5]:
def encode(recipe):
    x = np.zeros((1,len(ingredients)))
    indices = [dict_ingredients[ing] for ing in recipe]
    x[0,indices]=1
    return x
X = np.vstack([encode(recipes) for recipes in xtext])

This data set is quite voluminous and since our computational ressource are scarses we will remove the columns that corresponds to rarely used ingredients and select only the recipes of 10 types.

In [6]:
X.shape

(39774, 6714)

In [9]:
select_ingredients=ingredients[np.sum(X,axis=0)>100]
selected_type= np.array(['chinese', 'french', 'greek', 'indian', 'italian', 'jamaican',
       'korean', 'mexican', 'moroccan', 'thai'])
Xs=X[:,np.sum(X,axis=0)>100]
Xs=Xs[np.isin(y,selected_type),:]
ys=y[np.isin(y,selected_type)]

Our dataset is now ready to be processed. The final list of ingredients that we will used to recognize the cuisine types is :

In [10]:
Xs.shape

(27489, 640)

In [11]:
select_ingredients

array(['1% low-fat milk', 'Gochujang base', 'Italian bread',
       'Italian parsley leaves', 'Mexican cheese blend', 'Shaoxing wine',
       'Sriracha', 'Tabasco Pepper Sauce', 'Thai fish sauce',
       'Thai red curry paste', 'active dry yeast',
       'all purpose unbleached flour', 'all-purpose flour', 'allspice',
       'almond extract', 'almonds', 'ancho chile pepper',
       'anchovy fillets', 'andouille sausage', 'apple cider vinegar',
       'apples', 'arborio rice', 'artichoke hearts', 'arugula',
       'asian fish sauce', 'asparagus', 'avocado', 'baby spinach',
       'baby spinach leaves', 'bacon', 'bacon slices', 'baguette',
       'baking potatoes', 'baking powder', 'baking soda',
       'balsamic vinegar', 'bamboo shoots', 'bananas', 'barbecue sauce',
       'basil', 'basil leaves', 'basmati rice', 'bay leaf', 'bay leaves',
       'beansprouts', 'beef', 'beef broth', 'beef stock', 'beer', 'beets',
       'bell pepper', 'bittersweet chocolate', 'black beans',
       'blac

And the types of cuisines that we must recognize

In [12]:
selected_type

array(['chinese', 'french', 'greek', 'indian', 'italian', 'jamaican',
       'korean', 'mexican', 'moroccan', 'thai'], dtype='<U8')

## Build the training and test set

As usually, we will split the data into a training and a test dataset. the "train_test_split" function from scikit learn is dedicated to this task.

In [13]:
from sklearn.model_selection import train_test_split,cross_val_score


X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.3, random_state=0)

## Random Forest

We will use randomForest to solve the classification problem as a first try. We will fit a randomforest with 
 trees and estimate the accuracy of such classifier using cross_validation and the test set :

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix


clf = RandomForestClassifier(n_estimators=20)
scores = cross_val_score(clf, X_train, y_train, scoring='accuracy',cv=5)
scores.mean()  

0.787756182046907

We may also simply fit this model with the training data and produce prediction for the test set with the predict method.

In [11]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

And compute the accuracy with numpy or the built-in function of scikit learn

In [12]:
test_accuracy = np.sum(y_pred==y_test)/len(y_test)
test_accuracy

0.7696131926761246

In [13]:
test_accuracy =accuracy_score(y_test,y_pred)
test_accuracy

0.7696131926761246

Others metrics can be computed with the classification report tools from scikit learn:

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     chinese       0.78      0.83      0.80       822
      french       0.56      0.55      0.55       798
       greek       0.73      0.51      0.60       337
      indian       0.80      0.84      0.82       864
     italian       0.77      0.85      0.80      2399
    jamaican       0.74      0.34      0.46       161
      korean       0.69      0.56      0.62       232
     mexican       0.85      0.88      0.86      1931
    moroccan       0.73      0.46      0.57       260
        thai       0.83      0.65      0.73       443

    accuracy                           0.77      8247
   macro avg       0.75      0.65      0.68      8247
weighted avg       0.77      0.77      0.76      8247



and we may compute the detailled result with the confusion matrix

In [15]:
confusion_matrix(y_test,y_pred)

array([[ 681,   17,    1,   18,   24,    2,   33,   16,    0,   30],
       [  10,  436,   13,   15,  280,    2,    4,   29,    5,    4],
       [   4,   21,  171,   11,  103,    2,    0,   20,    5,    0],
       [  12,   18,    2,  730,   26,    1,    2,   54,   12,    7],
       [   3,  206,   40,   17, 2029,    3,    3,   83,   11,    4],
       [  15,   14,    0,   27,   18,   54,    4,   25,    1,    3],
       [  74,    1,    0,    1,    5,    1,  131,   14,    0,    5],
       [  19,   52,    5,   23,  103,    5,    4, 1705,   10,    5],
       [   1,   14,    3,   47,   43,    1,    1,   29,  120,    1],
       [  57,    6,    0,   25,   15,    2,    8,   40,    0,  290]])

## Questions?
### 1) What is the accuracy of this classifier on training data? What do you conclude?


In [16]:
accuracy_score(y_train,clf.predict(X_train))

0.9881509198628001

### 2) Try to improve the performance of this classifier in terms of accuracy ?
You can vary the number of trees between [20,50,100,200]. Use the GridSearchCV function seen in the first notebbok.

In [46]:
from sklearn.model_selection import GridSearchCV

### 3) Test another solution (Logistic regression)
You will be able to read the documentation of sklearn.linear_model.LogisticRegression

In [14]:
from sklearn.linear_model import LogisticRegression

### 4) Try to implement a rejection solution, to obtain an accuracy of at least 90%? 
To do this you will use the probabilities provided by the classifier and will only make a decision when these probabilities are above a certain threshold (to be determined). The predict_proba method should be able to help you.

## 5)  Try to solve the problem with more features with logistic regression.  Try to use the regularization parameter and compare the results of Multinomial  One-vs-Rest 